In [ ]:
#| default_exp gaussian

In [ ]:
#| include: false
from fastcore.all import *

In [ ]:
#| exporti
import gtsam
import numpy as np
import plotly.express as px
import pandas as pd

In [ ]:
#| export
def sample_conditional(node: gtsam.GaussianConditional, N: int, parents: list = [], sample: dict = {}):
    """Sample from conditional """
    # every node ~ exp(0.5*|R x + S p - d|^2)
    # calculate mean as inv(R)*(d - S p)
    d = node.d()
    n = len(d)
    rhs = d.reshape(n, 1)
    if len(parents) > 0:
        rhs = rhs - node.S() @ np.vstack([sample[p] for p in parents])
    # sample from conditional Gaussian
    invR = np.linalg.inv(node.R())
    return invR @ (rhs + np.random.normal(size=(n, N)))


def sample_bayes_net(bn: gtsam.GaussianBayesNet, N: int) -> dict:
    """ High performance ancestral sampling.
        It returns a dictionary of nj x N samples, where n_j is the dimensionality for key j.
    """
    sample = {}
    for i in reversed(range(bn.size())):
        conditional = bn.at(i)
        key, *parents = conditional.keys()
        sample[key] = sample_conditional(bn.at(i), N, parents, sample)
    return sample

In [ ]:
_x_, _y_ = 11, 12
bayesNet = gtsam.GaussianBayesNet()
I_1x1 = np.eye(1, dtype=float)
bayesNet.push_back(gtsam.GaussianConditional(_x_, [9.0], I_1x1, _y_, I_1x1))
bayesNet.push_back(gtsam.GaussianConditional(_y_, [5.0], I_1x1))

sample_bayes_net(bayesNet, 4)

{12: array([[2.83568484, 5.93936955, 3.94485894, 6.90187555]]),
 11: array([[5.2819049 , 2.60654641, 4.86926211, 0.530073  ]])}